## Example workbook for DIC <-> EBSD linking

In [ ]:
# Load third-party packages
import numpy as np
%matplotlib qt
import matplotlib.pyplot as plt

# Load quat, ebsd and hrdic packages from defdap package
from defdap.quat import Quat
import defdap.ebsd as ebsd
import defdap.hrdic as hrdic

### Load in and display DIC map

In [ ]:
#Load in DIC map
DicFilePath = "example_data/"
DicMap = hrdic.Map(DicFilePath, "B00005.txt")

#Set crop
DicMap.setCrop(xMin=15, xMax=28, yMin=30, yMax=15)

In [ ]:
DicMap.setScale(micrometrePerPixel=0.02)

In [ ]:
# Display max shear map
DicMap.plotMaxShear(plotColourBar=True, plotScaleBar=True, vmin=0, vmax=0.1)

### Print stats table

In [ ]:
DicMap.printStatsTable(percentiles=['Min', 5, 'Mean', 95,'Max'], components = ['mss', 'f11', 'f22'])

### Effective shear strain histogram

In [ ]:
histData = DicMap.crop(DicMap.max_shear).flatten()

fig, ax = plt.subplots(1)

counts, bins, bars = ax.hist(histData, bins=np.linspace(0,0.1,100), color='r', histtype='step')

ax.set_xlabel("Effective shear strain")
ax.set_ylabel("Frequency")

fig.show()

### Load in EBSD map then calculate local misorientation and display it

In [ ]:
# Load in EBSD map
EbsdFilePath = "example_data/Map Data 2-DIC area"
EbsdMap = ebsd.Map(EbsdFilePath, "cubic")

# Load in slip system definitions (optional)
EbsdMap.loadSlipSystems("cubic_fcc")

# Rotate the map 180 degrees if necessary
EbsdMap.transformData()

# Create the internal array of quaternions for orientation calculations
EbsdMap.buildQuatArray()

# Find boundaries with given misorientation tolerance (in degrees)
EbsdMap.findBoundaries(boundDef = 6)

# Find grains, minGrainSize is the minimum number of pixels for a grain
EbsdMap.findGrains(minGrainSize = 10)

# Calculate grain local misorientation (Grain Reference Orientation Deviation (GROD) w.r.t grain mean orientations)
EbsdMap.calcGrainMisOri(calcAxis = True)

In [ ]:
EbsdMap.plotEulerMap()

In [ ]:
EbsdMap.plotMisOriMap(plotGBs=True, vmin=0, vmax=4)

### Plot IPF of all grains mean orientation

In [ ]:
# Calculate grain mean orientations (but is already done here as part of calcGrainMisOri above)
# EbsdMap.calcGrainAvOris()

grainMeanOris = [grain.refOri for grain in EbsdMap]
refDir = np.array([1, 0, 0])

Quat.plotIPF(grainMeanOris, refDir, EbsdMap.crystalSym)

### Find a grain of interest (click around the map)

In [ ]:
EbsdMap.locateGrainID()

### Plot misorientation and IPF for isolated grain

In [ ]:
currEbsdGrain = EbsdMap[EbsdMap.currGrainId]

In [ ]:
currEbsdGrain.plotMisOri()

In [ ]:
currEbsdGrain.plotMisOri(component=3)

In [ ]:
plot = currEbsdGrain.plotOriSpread(direction=refDir, c='b', s=1, alpha=0.2)
currEbsdGrain.plotRefOri(direction=refDir, c='k', s=100, plot=plot)

### Plot phase map

In [ ]:
EbsdMap.plotPhaseMap()

## Link EBSD map to DIC map

### Define homologous points

In [ ]:
# Example of use of interactive tool coming soon...

In [ ]:
# Set homologous points   (x, y)
DicMap.homogPoints = np.array((
    (303, 408), (120, 392), (275, 25)
))

EbsdMap.homogPoints = np.array((
    (536, 776), (236, 790), (466, 160)
))

### Display map with homologous points

In [ ]:
EbsdMap.plotBoundaryMap()
plt.scatter(x=EbsdMap.homogPoints[:, 0], y=EbsdMap.homogPoints[:, 1], c='y', s=60)

In [ ]:
DicMap.plotMaxShear()
plt.scatter(x=DicMap.homogPoints[:, 0], y=DicMap.homogPoints[:, 1], c='y', s=60)

### Link the EBSD and DIC map

In [ ]:
DicMap.linkEbsdMap(EbsdMap)

### Plot max shear with boundaries

In [ ]:
DicMap.plotMaxShear(plotGBs=True, plotColourBar=True, plotScaleBar=True, vmin=0, vmax=0.1)

### Detect grains in the DIC map

In [ ]:
DicMap.findGrains(minGrainSize=10)

### Locate a grain of interest (click around)

In [ ]:
DicMap.locateGrainID(displaySelected=True)
# displaySelected will show the selcted grain in a separate figure window

### Plot max shear for isolated grain

In [ ]:
currDicGrain = DicMap[DicMap.currGrainId]

In [ ]:
currDicGrain.plotMaxShear()

### Plot a histogram of it

In [ ]:
plt.figure()

plt.hist(currDicGrain.maxShearList, bins=100, histtype='step');

plt.xlabel("Effective shear strain")
plt.ylabel("Frequency")

### Plot misorientation for isolated grain

In [ ]:
ebsdGrainId = DicMap.ebsdGrainIds[DicMap.currGrainId]
currEbsdGrain = EbsdMap[ebsdGrainId]

In [ ]:
currEbsdGrain.plotMisOri()

### Plot max shear for isolated grain with slip traces

In [ ]:
currDicGrain.calcSlipTraces()
currDicGrain.plotMaxShear(plotSlipTraces=True)